# Lenta.ru 

this is very old code

In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import time
import datetime
import pandas as pd
import pickle
import time

from tqdm import tqdm
from joblib import Parallel, delayed

# 1. Download hrefs

In [2]:
def monthlength(month,year):
    if year % 4 == 0:
         VisYear = 29
    else:
         VisYear = 28
    return [31,VisYear,31,30,31,30,31,31,30,31,30,31][month]

def days_of_year(year):
    dates = [ ]
    for mon in range(12):
        cur_month = [(year, mon+1, day) for day in range(1, monthlength(mon, year)+1)]
        dates.extend(cur_month)
    return dates 

def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) 

In [3]:
def news_data_get(year,month,day,rubrics='economics'):
    if len(str(day)) < 2:
        day = '0' + str(day)
        
    if len(str(month)) < 2:
        month = '0' + str(month)
        
    mainpage = 'https://lenta.ru/rubrics/{}/{}/{}/{}/'.format(rubrics, year, month, day)
    response = requests.get(mainpage)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    all_news = soup.find_all('div',{'class':'item news b-tabloid__topic_news'})
    return all_news

def news_to_df(news):
    new = { }
    new['date'] = news.find('span', {'class':'g-date item__date'}).text[8:]
    new['time'] = news.find('span', {'class':'time'}).text
    hr_tx = news.find('div', {'class':'titles'}).a
    new['title'] = hr_tx.text
    new['href'] = hr_tx.attrs['href']
    return new

In [13]:
# Финальные циклы (скачка с нуля)
# years = list(range(2019,2021))   # цикл пойдёт по годам и по рубрикам 
rubrics = ['russia', 'world', 'ussr', 'economics', 'forces', 'science', 'sport', 'culture', 'media']

# itog_titles = [ ]
# for yr in tqdm(years, desc='Years'):
#     for date in days_of_year(yr):
#         for rub in rubrics:
#             all_news = news_data_get(date[0],date[1],date[2],rub)
#             for new in all_news:
#                 cur_df = news_to_df(new)
#                 cur_df['rubrics'] = rub
#                 cur_df['year'] = date[0] 
#                 cur_df['month'] = date[1]
#                 cur_df['day'] = date[2]
#                 itog_titles.append(cur_df)

In [14]:
# Докачка
itog_titles = [ ]
days = days_of_year(2020)[152:300]

for date in tqdm(days):
    for rub in rubrics:
        all_news = news_data_get(date[0],date[1],date[2],rub)
        for new in all_news:
            cur_df = news_to_df(new)
            cur_df['rubrics'] = rub
            cur_df['year'] = date[0] 
            cur_df['month'] = date[1]
            cur_df['day'] = date[2]
            itog_titles.append(cur_df)

100%|██████████| 148/148 [1:02:05<00:00, 25.17s/it]


In [15]:
len(itog_titles)

29221

In [18]:
with open('news_data/2.lenta/lenta_titles_2020_new.pickle', 'wb') as f:
    pickle.dump(itog_titles, f)

In [19]:
df = pd.DataFrame(itog_titles)
print(df.shape)
df.head()

(29221, 8)


,date,time,title,href,rubrics,year,month,day
0,1 июня,02:31,Гидрометцентр рассказал о погоде в июне,/news/2020/06/01/june/,russia,2020,6,1
1,1 июня,08:00,Сбербанк напомнил россиянам о важности общения...,/news/2020/06/01/deti/,russia,2020,6,1
2,1 июня,10:01,Раскрыты планы начать голосование по Конституц...,/news/2020/06/01/golosovanie/,russia,2020,6,1
3,1 июня,10:38,Число умерших россиян с коронавирусом превысил...,/news/2020/06/01/cifra/,russia,2020,6,1
4,1 июня,11:13,Умер заразившийся коронавирусом митрополит РПЦ,/news/2020/06/01/mitropolit_umer/,russia,2020,6,1


# 2. Download news

In [20]:
def get_lenta_news(item_from_vect):
    url = 'https://lenta.ru' + item_from_vect['href']
    try:
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        txt = soup.find('div',{"class" : "b-text clearfix js-topic__text"}).text
        item_from_vect['text'] = txt
        item_from_vect['snippet'] = soup.find('meta', {'property' : "description"}).get('content')
        item_from_vect['title_image'] = soup.find('meta', {'property' : "og:image"}).get('content')
        return item_from_vect
    except:
        print(url)
        return { }

In [21]:
get_lenta_news(itog_titles[-10])

{'date': 'Сегодня',
 'time': '04:04',
 'title': 'Нурмагомедов сделал прогноз по\xa0лидерству россиян в\xa0UFC',
 'href': '/news/2020/10/25/habibb/',
 'rubrics': 'sport',
 'year': 2020,
 'month': 10,
 'day': 26,
 'text': 'Российский боец смешанного стиля (ММА) Хабиб Нурмагомедов сделал прогноз по возвращению лидерства соотечественниками в Абсолютном бойцовском чемпионате (UFC). Об этом он заявил в ходе общения с журналистами, передает ТАСС.По его мнению, спортсмены из России в скором времени вновь станут лидерами UFC. «У нас много молодых ребят, это подрастающее поколение. Скоро мы вернемся на вершину», — предрек он. Ранее представитель команды Нурмагомедова сообщил, что он будет наставником бойцов своего коллектива.Россиянин объявил об уходе из спорта 24 октября после того, как защитил титул чемпиона UFC в легком весе в бое с американцем Джастином Гейджи. Поединок, который возглавил турнир UFC 254 на бойцовском острове Яс в Абу-Даби, завершился во втором раунде, когда россиянин «задуши

In [22]:
from multiprocessing.pool import ThreadPool

def separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

def my_map(vect, parser_function):
    out = [parser_function(item) for item in vect]
    return(out)

def my_reduce(l):
    ll = [ ]
    for item in l:
        ll.extend(item)
    return(ll)

def mr_downloader(what, parts, parser_function):

    separatorlist = separator(what, parts)
    def mp(what):
        return my_map(what, parser_function=parser_function)

    pool = ThreadPool(parts)
    l = pool.map(mp, separatorlist)

    itog = my_reduce(l)
    return(itog)    

In [23]:
len(itog_titles)

29221

In [24]:
%%time
itog_news = mr_downloader(itog_titles, 20, parser_function=get_lenta_news)

https://lenta.ru/sport/2020/07/14/football-arsenal-liverpul/
https://lenta.ru/sport/2020/10/03/football-arsenal-sheffild-yunayted/
https://lenta.ru/sport/2020/10/03/football-brest-monako/
https://lenta.ru/sport/2020/10/03/football-chelsi-kristal-pelas/
https://lenta.ru/sport/2020/10/03/football-manchester-yunayted-tottenhem-hotspur/
https://lenta.ru/sport/2020/09/11/tennis-pablo-karreno-busta-ispaniya-aleksandr-zverev-germaniya/
https://lenta.ru/sport/2020/10/03/football-aston-villa-liverpul/
https://lenta.ru/sport/2020/09/11/football-ahmat-sochi/
https://lenta.ru/sport/2020/09/11/football-liverpul-lids-yunayted/
https://lenta.ru/sport/2020/09/04/football-niderlandy-polsha/
https://lenta.ru/sport/2020/09/04/hockey-ak-bars-vityaz/
https://lenta.ru/sport/2020/09/04/football-italiya-bosniya-i-gertsegovina/
https://lenta.ru/sport/2020/09/04/football-belarus-albaniya/
https://lenta.ru/sport/2020/09/04/basketball-mayami-hit-miluoki-baks/
https://lenta.ru/sport/2020/09/04/football-slovakiya-c

https://lenta.ru/sport/2020/09/21/football-vulverhempton-manchester-siti/
https://lenta.ru/news/2020/08/23/sergii/
https://lenta.ru/sport/2020/09/14/football-rostov-lokomotiv-m/
https://lenta.ru/sport/2020/07/16/football-everton-aston-villa/
https://lenta.ru/sport/2020/09/14/football-brayton-end-houv-albion-chelsi/
https://lenta.ru/sport/2020/09/14/football-paok-benfika/
https://lenta.ru/sport/2020/09/14/football-zenit-arsenal/
https://lenta.ru/sport/2020/09/14/football-sheffild-yunayted-vulverhempton/
https://lenta.ru/sport/2020/09/14/football-dinamo-k-az-alkmar/
https://lenta.ru/sport/2020/07/16/football-kristal-pelas-manchester-yunayted/
https://lenta.ru/sport/2020/07/16/football-ufa-dinamo-m/
https://lenta.ru/sport/2020/07/16/football-lester-siti-sheffild-yunayted/
https://lenta.ru/sport/2020/09/07/football-bosniya-i-gertsegovina-polsha/
https://lenta.ru/sport/2020/09/07/football-severnaya-irlandiya-norvegiya/
https://lenta.ru/sport/2020/09/07/football-kazahstan-belarus/
https://le

https://lenta.ru/sport/2020/09/16/football-vulverhempton-stok-siti/
https://lenta.ru/sport/2020/09/16/football-bavariya-shalke-04/
https://lenta.ru/sport/2020/08/03/basketball-los-andzheles-klippers-finiks-sanz/
https://lenta.ru/sport/2020/10/15/football-himki-spartak-m/
https://lenta.ru/sport/2020/10/15/basketball-tsska-alba/
https://lenta.ru/sport/2020/10/15/football-nim-pszh/
https://lenta.ru/sport/2020/10/15/football-chelsi-sautgempton/
https://lenta.ru/sport/2020/10/15/football-nyukasl-yunayted-manchester-yunayted/
https://lenta.ru/sport/2020/08/03/basketball-yuta-dzhaz-los-andzheles-leykers/
https://lenta.ru/sport/2020/08/03/football-kopengagen-istanbul-bashakshehir/
https://lenta.ru/sport/2020/10/15/basketball-barselona-panatinaikos/
https://lenta.ru/sport/2020/10/15/football-manchester-siti-arsenal/
https://lenta.ru/sport/2020/08/03/hockey-edmonton-oylerz-chikago-blekhouks/
https://lenta.ru/sport/2020/09/09/tennis-andrey-rublyov-rossiya-daniil-medvedev-rossiya/
https://lenta.ru

https://lenta.ru/news/2020/09/11/vse_budet_besplatno/
https://lenta.ru/sport/2020/07/21/football-rostov-zenit/
https://lenta.ru/sport/2020/07/21/football-tsska-tambov/
https://lenta.ru/sport/2020/10/25/football-arsenal-lester-siti/
https://lenta.ru/sport/2020/10/18/football-tsska-dinamo-m/
https://lenta.ru/sport/2020/10/18/football-lester-siti-aston-villa/
https://lenta.ru/sport/2020/10/25/football-ahmat-ufa/
https://lenta.ru/sport/2020/10/25/football-lion-monako/
https://lenta.ru/sport/2020/10/18/football-monako-monpelie/
https://lenta.ru/sport/2020/10/10/football-irlandiya-uels/
https://lenta.ru/sport/2020/10/18/football-rostov-ahmat/
https://lenta.ru/news/2020/09/04/ugrosa/
https://lenta.ru/sport/2020/10/25/football-sautgempton-everton/
https://lenta.ru/sport/2020/10/25/football-vulverhempton-nyukasl-yunayted/
https://lenta.ru/sport/2020/10/18/football-kristal-pelas-brayton-end-houv-albion/
https://lenta.ru/sport/2020/10/18/football-tottenhem-hotspur-vest-hem-yunayted/
https://lenta

In [25]:
len(itog_news)

29221

In [26]:
itog_news = [itog for itog in itog_news if len(itog.keys()) != 0 ]
len(itog_news)

28818

In [27]:
len(set([item['href'] for item in itog_news]))

28740

In [28]:
with open('news_data/2.lenta/lenta_news_2020_new.pickle', 'wb') as f:
    pickle.dump(itog_news, f)